In [1]:
from utils import *

Using TensorFlow backend.


In [2]:
path = 'data/mnist/'
# path = 'data/mnist/sample' # sample path

In [3]:
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')

In [4]:
train = train.values
test = test.values
# 打乱
train = np.random.permutation(train)

In [5]:
valid = train[:2000, :]
valid_data = valid[:, 1:]
valid_label = onehot(valid[:, 0])

In [6]:
train = train[2000:, :]

In [7]:
train_data = train[:, 1:]
train_label = onehot(train[:, 0])

In [8]:
train_data.shape, train_label.shape, valid_label.shape, valid_data.shape

((40000, 784), (40000, 10), (2000, 10), (2000, 784))

In [9]:
def reshape(data, target_size): return np.reshape(data, target_size)

In [10]:
train_data = reshape(train_data, [40000, 1, 28, 28])
valid_data = reshape(valid_data, [2000, 1, 28, 28])
train_data.shape, train_label.shape, valid_label.shape, valid_data.shape

((40000, 1, 28, 28), (40000, 10), (2000, 10), (2000, 1, 28, 28))

In [11]:
mean_px = train_data.mean().astype(np.float32)
std_px = train_data.std().astype(np.float32)

In [12]:
def norm_input(x): return (x-mean_px)/std_px

In [14]:
BATCH_SIZE = 64

# 线性模型

In [14]:
def get_lin_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [15]:
lm = get_lin_model()

In [18]:
gen = image.ImageDataGenerator()
batches = gen.flow(train_data, train_label, batch_size=BATCH_SIZE)
val_batches = gen.flow(valid_data, valid_label, batch_size=BATCH_SIZE)

In [17]:
lm.fit_generator(
    batches, 
    samples_per_epoch=len(train_data) / 20, 
    nb_epoch=1, 
    validation_data=val_batches,
    nb_val_samples=len(valid_data) / 20
)

Epoch 1/1
2000/2000 [==============================] - 1s - loss: 1.4026 - acc: 0.5315 - val_loss: 0.7518 - val_acc: 0.7667


In [18]:
lm.fit_generator(
    batches, 
    samples_per_epoch=len(train_data) / 20, 
    nb_epoch=1, 
    validation_data=val_batches,
    nb_val_samples=len(valid_data) / 20
)

Epoch 1/1
2000/2000 [==============================] - 0s - loss: 0.6715 - acc: 0.7840 - val_loss: 0.5619 - val_acc: 0.8417


# 多层神经网络

In [19]:
def get_fc_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(lr=.01), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [20]:
fc = get_fc_model()

In [21]:
fc.fit_generator(
    batches, 
    samples_per_epoch=batches.n, 
    nb_epoch=1, 
    validation_data=val_batches,
    nb_val_samples=val_batches.n
)

Epoch 1/1
40000/40000 [==============================] - 6s - loss: 0.8576 - acc: 0.8798 - val_loss: 0.1976 - val_acc: 0.9455


In [22]:
def get_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [23]:
model = get_model()

In [24]:
model.fit_generator(
    batches, 
    samples_per_epoch=batches.n, 
    nb_epoch=1, 
    validation_data=val_batches,
    nb_val_samples=val_batches.n
)

Epoch 1/1
40000/40000 [==============================] - 15s - loss: 0.1375 - acc: 0.9565 - val_loss: 0.0494 - val_acc: 0.9855


# 数据增强

In [25]:
model = get_model()

In [26]:
gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(train_data, train_label, batch_size=BATCH_SIZE)
val_batches = gen.flow(valid_data, valid_label, batch_size=BATCH_SIZE)

In [27]:
model.fit_generator(
    batches, 
    samples_per_epoch=batches.n, 
    nb_epoch=1, 
    validation_data=val_batches,
    nb_val_samples=val_batches.n
)

Epoch 1/1
40000/40000 [==============================] - 15s - loss: 0.2505 - acc: 0.9200 - val_loss: 0.0979 - val_acc: 0.9680


In [30]:
def get_model_bn_do():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
#         BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [31]:
model = get_model_bn_do()

In [94]:
model.optimizer.lr=0.01

In [32]:
model.fit_generator(
    batches, 
    samples_per_epoch=batches.n, 
    nb_epoch=1, 
    validation_data=val_batches,
    nb_val_samples=val_batches.n
)

Epoch 1/1
40000/40000 [==============================] - 43s - loss: 0.2692 - acc: 0.9163 - val_loss: 0.2043 - val_acc: 0.9395.2718 - 


In [139]:
model.optimizer.lr=0.001

In [33]:
model.fit_generator(
    batches, 
    samples_per_epoch=batches.n, 
    nb_epoch=1, 
    validation_data=val_batches,
    nb_val_samples=val_batches.n
)

Epoch 1/1
40000/40000 [==============================] - 43s - loss: 0.1103 - acc: 0.9651 - val_loss: 0.1041 - val_acc: 0.9685


In [34]:
model.optimizer.lr=0.01

In [35]:
model.fit_generator(
    batches, 
    samples_per_epoch=batches.n, 
    nb_epoch=1, 
    validation_data=val_batches,
    nb_val_samples=val_batches.n
)

Epoch 1/1
40000/40000 [==============================] - 43s - loss: 0.0873 - acc: 0.9725 - val_loss: 0.0621 - val_acc: 0.9810


In [36]:
model.optimizer.lr=0.1

In [38]:
model.fit_generator(
    batches, 
    samples_per_epoch=batches.n, 
    nb_epoch=4, 
    validation_data=val_batches,
    nb_val_samples=val_batches.n
)

Epoch 1/4
40000/40000 [==============================] - 42s - loss: 0.0674 - acc: 0.9784 - val_loss: 0.0559 - val_acc: 0.9845
Epoch 2/4
40000/40000 [==============================] - 42s - loss: 0.0629 - acc: 0.9804 - val_loss: 0.0444 - val_acc: 0.9855
Epoch 3/4
40000/40000 [==============================] - 42s - loss: 0.0571 - acc: 0.9821 - val_loss: 0.0448 - val_acc: 0.9880
Epoch 4/4
40000/40000 [==============================] - 43s - loss: 0.0579 - acc: 0.9816 - val_loss: 0.0373 - val_acc: 0.9890


In [43]:
model.optimizer.lr=0.001

In [50]:
model.fit_generator(
    batches, 
    samples_per_epoch=batches.n, 
    nb_epoch=3, 
    validation_data=val_batches,
    nb_val_samples=val_batches.n
)

Epoch 1/3
40000/40000 [==============================] - 42s - loss: 0.0472 - acc: 0.9855 - val_loss: 0.0452 - val_acc: 0.9855
Epoch 2/3
40000/40000 [==============================] - 42s - loss: 0.0445 - acc: 0.9866 - val_loss: 0.0375 - val_acc: 0.9875
Epoch 3/3
40000/40000 [==============================] - 43s - loss: 0.0413 - acc: 0.9875 - val_loss: 0.0333 - val_acc: 0.9880


In [51]:
model.optimizer.lr=0.0003

In [ ]:
model.fit_generator(
    batches, 
    samples_per_epoch=batches.n, 
    nb_epoch=1, 
    validation_data=val_batches,
    nb_val_samples=val_batches.n
)

Epoch 1/1
 5760/40000 [===>..........................] - ETA: 36s - loss: 0.0473 - acc: 0.9852

In [45]:
preds = model.predict(reshape(test, [28000, 1, 28, 28]))

In [46]:
pre = np.argmax(preds, 1)

In [47]:
conv_y_submission = pd.DataFrame({
        'ImageId': range(1, 28001), 
        'Label': pre 
    })

In [49]:
conv_y_submission.to_csv('./conv_submission2.csv', index=False)